### Agentic RAG
 : 질문에 따라 문서를 검색하여 답변(RAG)하거나, 인터넷 검색 도구(Tavily Search:)를 활용하여 답변하는 에이전트

**참고**

- **Agentic RAG**  : RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행하는 에이전트

LangChain에서는 에이전트가 여러 툴을 조합하여 복잡한 문제를 해결하는 능력을 가지고 있기 때문에, Tool은 에이전트가 활용할 수 있는 기능적 요소
Agent 가 활용할 도구를 정의하여 Agent가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있음

-  **도구(Tools)** :특정 작업(계산, 검색, API 호출 등)을 수행하는 도구
- **에이전트** : 사용자의 요청을 분석하고 적절한 툴을 사용하여 문제를 해결하는 주체


### 웹 검색도구: Tavily Search

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

`.env`에 환경변수 등록

- `TAVILY_API_KEY=발급 받은 Tavily API KEY 입력`

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력
logging.langsmith("Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
Agents


In [3]:
import os

os.environ['LANGCHAIN_PROJECT'] = 'Agents'

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성
# k=6은 검색 결과를 6개까지 가져오겠다는 의미
search = TavilySearchResults(k=6)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행


In [5]:
# 검색 결과
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'url': 'https://www.youtube.com/watch?v=BpynjxkklPc',
  'content': '... 카카오프렌즈 판교아지트 ✓주소 : 경기 성남시 분당구 판교역로 166 1층 ✓전화번호 : 031-601-7225 ✓영업시간 : 평일 10시 ~ 21시 / 주말공휴일 12\xa0...'},
 {'url': 'https://www.instagram.com/kakaofriends_official/p/Ckr_zDmNlEn/',
  'content': "카카오프렌즈 '판교 아지트점' OPEN ✨ ... (경기 성남시 분당구 판교역로 166) #카카오프렌즈 #Kakaofriends #카카오프렌즈스토어 #판교아지트 #카카오\xa0..."},
 {'url': 'https://www.instagram.com/kakaofriends_official/reel/Ck0JcHGJ7WP/',
  'content': '📍 위치: 판교역 4번 출구 “카카오 판교아지트” 1층 (경기 성남시 분당구 판교역로 166) #카카오프렌즈 #Kakaofriends #카카오프렌즈스토어 #판교 #카카오\xa0...'},
 {'url': 'https://www.kakaocorp.com/ir/wayToCome',
  'content': '고객센터. 주소: 경기도 성남시 분당구 판교역로 241번길 20 미래에셋 벤처타워 1층 (우)13494. 전화: 1577-3754(카카오 서비스) (유료) 1577-3321(다음 서비스) (유료).'},
 {'url': 'https://m.blog.naver.com/hj961030/222923648353',
  'content': '직접 보면 더 예쁜 카카오프렌즈 판교 아지트점\n\u200b\n모두 모두 카카오프렌즈 판교 아지트점에서 만나요-!\n경기도 성남시 분당구 판교역로 166\n\u200b\n♥n2u1_늘이의 여행일기♥\n이 블로그\n경기도\n카테고리 글\n카테고리\n이 블로그\n경기도\n카테고리 글 8. 21:39\n카카오프렌즈 판교 아

### 문서 기반 문서 검색 도구: Retriever

내가 넣은 문서에 대해 조회를 수행할 retriever도 생성

**실습에 활용한 문서**

초보 투자자를 위한 증권과 투자 따라잡기.pdf

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import JSONLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("초보 투자자를 위한 증권과 투자 따라잡기.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()

 `retriever` 객체의 `invoke()` 를 사용하여 사용자의 질문에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용


In [7]:
# 문서에서 관련성 높은 문서를 가져옴
retriever.invoke("채권에 대한 개념을 알려줘.")

[Document(metadata={'source': '초보 투자자를 위한 증권과 투자 따라잡기.pdf', 'page': 29}, page_content='채권시장의 이해 초보 투자자를 위한 증권과 투자 따라잡기056 057\u2002기타 채권\n1) 주식관련채권\n채권의 보유자에게 일정한 시기에 일정한 가격으로 발행기업의 증권을  \n매입하거나, 혹은 다른 형태의 증권과 교환할 수 있는 권리 (option)가 부여된 \n채권입니다. 신주인수권부사채 (BW; bond with warrant) 는 채권 보유자\n에게 일정요건이 충족될 때 일정수 또는 금액에 해당하는 신주를 매입할 \n수 있는 권리가 부여된 채권입니다. 이에 반해 전환사채 (CB; convertible \nbond)는 채권에서 발행회사의 주식으로 전환할 수 있는 권리가 부여된 채\n권입니다. 신주인수권부사채나 전환사채는 보통사채에 비해 발행금리가 \n낮아 기업의 입장에서는 적은 비용으로 자금을 조달할 수 있는 장점이 있\n습니다. 투자자의 입장에서는 주가상승 시 매매차익을 올릴 수 있다는 투\n자의 매력이 있습니다. 교환사채 (EB; exchange bond) 는 채권을 발행한 회\n사가 보유하고 있는 다른 회사의 주식이나 유가증권과 교환을 청구할 수 \n있는 권리가 부여된 채권입니다.\n2) 자산유동화증권(ABS; asset backed securities)\n금융기관이나 기업 등이 보유하고 있는 자산을 담보로 발행하여 제3자\n에게 매각하는 증권을 말합니다. 초기 형태로는 주택저당대출을 대상으로 \n한 주택저당담보부채권 (MBS; mortgage backed securities) 이 있습니다. \n이후 다른 자산에까지 확대 적용되면서 최근에는 자동차할부금융‧신용\n카드대출‧리스대출 등을 대상으로 한 담보채권을 포함합니다.\n3) 국제채(international bond)\n국내에서 발행되는 국내채에 대응되는 개념으로 외국에서 유통되거나 \n외화로 표시되어 있는 채권을 지칭합니다. 국제채는 외국

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환 가능


`create_retriever_tool` 함수로 `retriever` 를 도구로 변환

In [8]:
from langchain.tools.retriever import create_retriever_tool


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야함!!
)

### Agent 가 사용할 도구 목록 정의

Agent 가 사용할 도구 목록:  
  
`tools` 리스트는 `search`와 `retriever_tool`을 포함

In [9]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

#### Agent 생성

Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Tool Calling Agent 를 생성

In [11]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

마지막으로, 생성한 `agent` 를 실행하는 `AgentExecutor` 를 생성
- (참고)`verbose=False` 로 설정하여 중간 단계 출력을 생략

In [12]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

#### 에이전트 실행하기

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의 == 이전 상호작용을 기억하지 못함


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리

In [13]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [14]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'url': 'https://realjace.tistory.com/entry/2024-프로야구-포스트시즌-플레이오프-한국시리즈-일정-총정리', 'content': "2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 플레이오프 한국시리즈 일정 총정리 2024 프로야구 포스트시즌 경기방식 2024 프로야구 포스트시즌 일정 2024 프로야구 포스트시즌 경기방식 와일드카드 결정전에서 승리한 팀과 정규시즌 3위 팀이 맞붙습니다. 준플레이오프에서 승리한 팀과 정규시즌 2위 팀이 경기를 펼치게 됩니다. 플레이오프 승리팀과 정규시즌 1위 팀이 7전 4선 승제로 경기를 펼치게 됩니다. 2024 프로야구 포스트시즌 진출팀 특히 4위부터 6위까지 승차가 거의 없기 때문에 마지막까지 흥미진진할 것으로 보이는데요, 각 팀이 144경기를 소화해야 하며 9월 28일까지 정규시즌이 마무리될 것으로 보입니다. 2024 프로야구 포스트시즌 일정 플레이오프 1차전 플레이오프 2차전 플레이오프 3차전 플레이오프 4차전 '스포츠&게임 정보' 카테고리의 다른 글 2024 프로야구 포스트시즌 티켓 예매방법\xa0\xa0(3) 2024 프로야구 올스타전 투표결과 경기일정 티켓 예매방법 가격 총정리\xa0\xa0(0) 2024 프로야구 포스트시즌 티켓 예매방법 티스토리 홈 이동"}, {'url': 'https://m.blog.naver.com/hoga1201/223571117829', 'content': '2024년 프로야구 포스트시즌 가을야구 플레이오프 진출팀 예상 분석 상대팀 별 잔여경기 : 네이버 블로그 2024년 프로야구 포스트시즌 가

`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공


In [15]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "선물과 주식에 대한 차이점을 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 선물과 주식의 차이점
Log: 
Invoking: `pdf_search` with `{'query': '선물과 주식의 차이점'}`



[관찰 내용]
Observation: 대부분의 개인투자자들은 채권을 만기까지 보유하는 경우가 많기 때문에 채권투자 시에는 신용등급을 꼼꼼하게 
살피는 것이 무엇보다 중요합니다.
자료 : 시민을 위한 증권투자 이야기(증권선물거래소)유가증권의 대표 증권인 주식과 채권은 같은 자본증권이지만 기업과 
투자자의 입장에서 보면 여러 가지 면에서 다른 특징을 보이고 있습니다.4│채권과 주식의 차이
◆ 주식과 채권의 차이  ◆
구 분 주식 채권
자본의 성격 자기자본 타인자본
발행자 주식회사 정 부‧지자체‧특 별법인‧주식회사
소유자의 지위 주주 채권자
경영참가 있음 없음
존속기간 영구적 한시적
이익형태 및 성격 배당‧ 가변적 이자‧확정적

파생상품시장의 이해 초보 투자자를 위한 증권과 투자 따라잡기068 069선물거래는 선도거래와 마찬가지로 계약자 간에 임의로 행해지는 사적
인 계약을 말합니다. 실생활에서 볼 수 있는 전형적인 선도거래로는 배추
나 무 등 밭 전체의 농작물을 미리 사거나 파는 “밭떼기” 라는 거래를 들 
수 있습니다. 즉, 배추 혹은 무의 씨앗을 뿌릴 때 정해진 가격으로 밭 전체
에서 수확될 배추 혹은 무를 사고팔 것을 계약하는 것입니다. 실제로 돈과 
물건(배추, 무 등) 을 주고 받는 시점은 배추, 무 등이 다 자라나 수확되는 시
기가 될 것입니다.
이처럼 선물거래 (Futures)  혹은 선도거래 (Forward) 는 모든 거래조건
을 현재시점에서 계약하고 상품의 인수도와 대금결제는 미래 일정시점에
서 이루어지는 거래를 말합니다.
다만, 양 거래당사자의 계약인 선도거래와 달리 선물거래는 조직화된 
장소인 거래소에 서 특정 상품을 현재시점에서 정한 가격으로 미래 일정시1│선물의 개념
선물시장의 이해
점에 인수도할 것을 약속하는 거래를 말합니다. 선물거래는 선도거래와 

## 이전 대화내용 기억하는 Agent

`RunnableWithMessageHistory`: 이전의 대화내용을 기억하기 위해 사용  

`AgentExecutor`를 `RunnableWithMessageHistory`로 감싸줌

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [17]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "선물과 주식에 대한 차이점을 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 선물과 주식의 차이점
Log: 
Invoking: `pdf_search` with `{'query': '선물과 주식의 차이점'}`



[관찰 내용]
Observation: 대부분의 개인투자자들은 채권을 만기까지 보유하는 경우가 많기 때문에 채권투자 시에는 신용등급을 꼼꼼하게 
살피는 것이 무엇보다 중요합니다.
자료 : 시민을 위한 증권투자 이야기(증권선물거래소)유가증권의 대표 증권인 주식과 채권은 같은 자본증권이지만 기업과 
투자자의 입장에서 보면 여러 가지 면에서 다른 특징을 보이고 있습니다.4│채권과 주식의 차이
◆ 주식과 채권의 차이  ◆
구 분 주식 채권
자본의 성격 자기자본 타인자본
발행자 주식회사 정 부‧지자체‧특 별법인‧주식회사
소유자의 지위 주주 채권자
경영참가 있음 없음
존속기간 영구적 한시적
이익형태 및 성격 배당‧ 가변적 이자‧확정적

파생상품시장의 이해 초보 투자자를 위한 증권과 투자 따라잡기068 069선물거래는 선도거래와 마찬가지로 계약자 간에 임의로 행해지는 사적
인 계약을 말합니다. 실생활에서 볼 수 있는 전형적인 선도거래로는 배추
나 무 등 밭 전체의 농작물을 미리 사거나 파는 “밭떼기” 라는 거래를 들 
수 있습니다. 즉, 배추 혹은 무의 씨앗을 뿌릴 때 정해진 가격으로 밭 전체
에서 수확될 배추 혹은 무를 사고팔 것을 계약하는 것입니다. 실제로 돈과 
물건(배추, 무 등) 을 주고 받는 시점은 배추, 무 등이 다 자라나 수확되는 시
기가 될 것입니다.
이처럼 선물거래 (Futures)  혹은 선도거래 (Forward) 는 모든 거래조건
을 현재시점에서 계약하고 상품의 인수도와 대금결제는 미래 일정시점에
서 이루어지는 거래를 말합니다.
다만, 양 거래당사자의 계약인 선도거래와 달리 선물거래는 조직화된 
장소인 거래소에 서 특정 상품을 현재시점에서 정한 가격으로 미래 일정시1│선물의 개념
선물시장의 이해
점에 인수도할 것을 약속하는 거래를 말합니다. 선물거래는 선도거래와 

In [18]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous answer into English:

1. **Transaction Type**:
   - **Stocks**: Stocks represent ownership in a company, and shareholders have the right to participate in management.
   - **Futures**: Futures trading is a contract to buy or sell a specific commodity at a predetermined price at a future date. This occurs on organized exchanges.

2. **Ownership**:
   - **Stocks**: Holding stocks means owning a part of the company, and shareholders can receive dividends.
   - **Futures**: A futures contract does not transfer ownership of the actual asset; rather, it is a contract regarding future price fluctuations.

3. **Duration**:
   - **Stocks**: Stocks can be held indefinitely.
   - **Futures**: Futures contracts have an expiration date, and the contract ends at a specific point in time.

4. **Profit Type**:
   - **Stocks**: The profit from stocks is variable and comes in the form of dividends.
   - **Futures**: The profit from futures is determined by

## Agent 템플릿

In [20]:
# PyMuPDF : 텍스트, 이미지, 주석, 레이아웃 등 PDF의 모든 요소 처리 가능한 패키지
!pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 30.1 MB/s eta 0:00:0000:0100:01m


In [21]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성
# k=6은 검색 결과를 6개까지 가져오겠다는 의미
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("./초보 투자자를 위한 증권과 투자 따라잡기.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가 ###
# tools 리스트에 search와 retriever_tool을 추가
tools = [search, retriever_tool]

########## 2. LLM 을 정의  ##########
# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의##########

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########

# 에이전트를 생성
# llm, tools, prompt를 인자로 사용
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 0 of document ./초보 투자자를 위한 증권과 투자 따라잡기.pdf
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 1 of document ./초보 투자자를 위한 증권과 투자 따라잡기.pdf
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 72 of document ./초보 투자자를 위한 증권과 투자 따라잡기.pdf
  warnings.warn(


In [22]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input":  "국내 증권 시장의 역사를 문서에서 찾아서 추천해줘."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 국내 증권 시장의 역사
Log: 
Invoking: `pdf_search` with `{'query': '국내 증권 시장의 역사'}`



[관찰 내용]
Observation: 지정한 종목을 코스닥증권시장(주) 호가중개시스템을 통해 거래할 수 있
게 하였습니다. 증권거래소와 코스닥시장에 이어 2000년 3월 27일 세 번
째로 문을 열었다는 뜻에서 제3시장이라 하였습니다. 그리고 2005년 7월 
13일 제3시장이 제 기능을 못한다는 지적에 따라 증권업협회가 제도를 손
질해 프리보드(Free Board)로 새롭게 출범하게 되었습니다. 그 후 증권업
협회, 선물협회, 자산운용협회가 합병하여 한국금융투자협회가 출범하면
서 2014년 6월 17일, 프리보드를 전면 개편하여 K-OTC(Korea Over-The-
Counter)로 명칭을 변경하였습니다.
벤처기업의 산실 NASDAQ
장외시장 중 세계에서 가장 발전된 시장이 미국증권업협회가 주관하고 있는 NASDAQ(National Association 
of Securities Dealers Automated Quotations) 시장입니다. NASDAQ 시장은 미국증권업협회가 전산망을 
통해 전국적으로 통합한 시장으로서 1971년 2월 개설되었습니다. 하지만 80년 후반에 들어와 거래규모가 커졌고 
NASDAQ 시장의 주종을 이루고 있던 IT 등 첨단산업이 급속히 성장하면서 NASDAQ 시장은 세계 최고의 증권
시장인 뉴욕증권거래소(NYSE)에 필적하는 새로운 형태의 시장조직으로 발전하였습니다. 즉, 거래량을 기준으로 
할 때 뉴욕증권거래소에 이어 세계에서 두 번째로 큰 시장입니다. 현재 NASDAQ 시장에 상장되어 있는 대표적인 
기업으로는 마이크로소프트(MS), 아마존, 애플, 구글 등을 들 수 있습니다. 우리나라의 기업들도 일부 NASDAQ 
시장에 상장되어 있으며, 최근에는 중국 기업들이 러시를 이루고 있습니다.
우리나라의 증권 시장은 2005년에 새로

In [23]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
The history of the domestic securities market has undergone several changes. On March 27, 2000, the third market was opened, and on July 13, 2005, the third market was relaunched as the Free Board. Later, on June 17, 2014, it was renamed K-OTC (Korea Over-The-Counter).

In 2005, recognizing the limitations of continuous growth and development due to the overlapping market operation system, the Korean securities market underwent changes to prepare for the trend of exchange integration and international competition among markets.

In the 1970s, construction stocks were prominent; in the 1980s, stocks related to high-tech industries such as electronics and automobiles gained attention. In the 1990s, with the allowance of direct foreign investment, industry-leading stocks emerged, driven by foreign investors. In the 2000s, with the development of the IT industry, internet and telecommunications-related stocks led the market. Around 2020, with the advent of the Fourth Industrial Rev

In [24]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "넷플릭스 TV 프로그램 흑백요리사 최종 우승자를 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 넷플릭스 흑백요리사 최종 우승자
Log: 
Invoking: `tavily_search_results_json` with `{'query': '넷플릭스 흑백요리사 최종 우승자'}`



[관찰 내용]
Observation: [{'url': 'https://www.mk.co.kr/news/culture/11134868', 'content': "흑수저 '나폴리 맛피아' 권성준(30)이 '흑백요리사' 최종 우승을 거머쥐었다. 8일 오후 공개된 넷플릭스 오리지널 예능 '흑백요리사 : 요리 계급 전쟁(이하 '흑백요리사') 최종 11~12회에서 나폴리 맛피아가 백수저 에드워드 리를 꺾고 우승, 상금 3억원을 차지했다."}, {'url': 'https://v.daum.net/v/20241008180911712', 'content': '8일 공개된 넷플릭스 \'흑백요리사: 요리 계급 전쟁\'에서는 \'나폴리맛피아\' 권성준이 백수저 에드워드리를 꺾고 최종 우승을 거머쥐었다. 권성준은 우승자 발표 하루 전 열린 \'흑백요리사\' 톱8 간담회에서 "시즌2에 섭외가 온다면 백수저로 (섭외가) 올 것 같다.'}, {'url': 'https://www.chosun.com/culture-life/food-taste/2024/10/11/DSMXC57PSJDBFDSDDZVHNCS4KY/', 'content': '흑백요리사 우승자 식당에 11만명 몰려 150만원 암표까지 넷플릭스 요리 서바이벌 흑백요리사의 최종 우승자인 나폴리 맛피아29·권성준가 자신이 운영하는 식당 예약을 암표 거래하는 사람들에게 경고의 메시지를 날렸다. 나폴리 맛피아는 지난 8일 공개된 흑백요리사 최종..'}, {'url': 'https://namu.wiki/w/흑백요리사:+요리+계급+전쟁/진행+결과', 'content': '최종 우승자. 권성준(나폴리 맛피아) ... 넷플릭스 코리아 채널에 생존한 흑수저들 중에서 편집된 참가자들의 미

In [25]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
넷플릭스 '흑백요리사: 요리 계급 전쟁'에서 '나폴리 맛피아' 권성준 셰프가 최종 우승을 차지했습니다! 축하합니다! 🎉 #흑백요리사 #권성준 #넷플릭스


In [26]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 우승자의 소감도 찾아줘."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 흑백요리사 권성준 우승 소감
Log: 
Invoking: `tavily_search_results_json` with `{'query': '흑백요리사 권성준 우승 소감'}`



[관찰 내용]
Observation: [{'url': 'https://redduck.tistory.com/entry/%ED%9D%91%EB%B0%B1%EC%9A%94%EB%A6%AC%EC%82%AC-%EB%82%98%ED%8F%B4%EB%A6%AC-%EB%A7%9B%ED%94%BC%EC%95%84-%EA%B6%8C%EC%84%B1%EC%A4%80-%EC%B5%9C%EC%A2%85-%EC%9A%B0%EC%8A%B9-%EC%9A%94%EB%A6%AC-%EA%B3%84%EA%B8%89-%EC%A0%84%EC%9F%81-%EB%B0%B1%EC%A2%85%EC%9B%90-%EC%95%88%EC%84%B1%EC%9E%AC', 'content': '4 days ago · - 소감: 권성준은 SNS를 통해 우승 소감을 전하며 기쁨을 표현함. 결론 "흑백요리사"는 흑수저 셰프들이 백수저 셰프들과의 경쟁을 통해 요리의\xa0...'}, {'url': 'http://www.msn.com/ko-kr/news/other/%ED%9D%91%EB%B0%B1%EC%9A%94%EB%A6%AC%EC%82%AC-%EC%9A%B0%EC%8A%B9-%EA%B6%8C%EC%84%B1%EC%A4%80-%EB%8F%8C%EC%97%B0-%EB%B0%98%EC%84%B1%EB%AC%B8-%EC%99%9C/ar-AA1rVoNr?apiversion=v2&noservercache=1&domshim=1&renderwebcomponents=1&wcseo=1&batchservertelemetry=1&noservertelemetry=1', 'content': "2 days ago · 넷플릭스 '흑백요리사: 요리 계급 전쟁'(이하 '흑백요리사') 에 출연한 

#### [실습] PDF를 활용한 RAG + 실시간 웹 검색을 활용한 챗봇 구축
 - 자유주제 가능
  
 - (예시) 요리 레시피 검색 및 추천 에이전트  
    목표: PDF로 제공된 요리책에서 원하는 레시피를 검색하고, 찾을 수 없으면 웹에서 정보를 검색해 추천 요리법을 제공하는 시스템을 구축
    
 - (예시) 여행 정보 검색 및 추천 에이전트  
   목표: PDF 문서로 제공된 여행 가이드북에서 특정 목적지에 대한 정보를 검색하고, 부족한 정보는 웹에서 검색해 종합적인 여행 계획을 제시하는 시스템을 구축

 - (예시) 논문 검색 및 관련 연구 서베이 에이전트
   목표: 공모전 혹은 프로젝트에 사용할 딥러닝 모델을 논문 RAG 문서로 활용하여, 논문 질의 응답을 받은 후 부족한 정보다 비교 모델을 웹에서 검색해 사용할 모델들을 정리해주는 시스템을 구축

In [27]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성
# k=6은 검색 결과를 6개까지 가져오겠다는 의미
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("/home/elicer/Agent RAG/(2회차) K-ICT 빅데이터센터_딥러닝교육_초급(0830).pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가 ###
# tools 리스트에 search와 retriever_tool을 추가
tools = [search, retriever_tool]

########## 2. LLM 을 정의  ##########
# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의##########

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########

# 에이전트를 생성
# llm, tools, prompt를 인자로 사용
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

In [29]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input":  "딥러닝 모델을 문서에서 찾아서 추천해줘. 없으면 웹에서 검색해서 추천해줘"},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 딥러닝 모델
Log: 
Invoking: `pdf_search` with `{'query': '딥러닝 모델'}`



[관찰 내용]
Observation: ai dentify 
딥러닝 개요

ai dentify 
딥러닝 프레임워크 개요

ai dentify 
머신러닝 개요

ai dentify 
• 딥러닝 학습 절차 
딥러닝이란 
데이터 로드 
네트워크 정의 
훈련하기 
평가하기 
훈련 데이터셋 
(입력, 레이블) 
평가 데이터셋 
(입력, 레이블) 
손실 함수 
네트워크 모델 
손실 
파라미터 업데이트 
feed forward 
back propagation 
모델의 출력값 y_와 실제 레이블 y간의 
차이 계산 
훈련된 네트워크 모델 
평가 정확도
[도구 호출]
Tool: tavily_search_results_json
query: 딥러닝 모델 추천
Log: 
Invoking: `tavily_search_results_json` with `{'query': '딥러닝 모델 추천'}`



[관찰 내용]
Observation: [{'url': 'https://lsjsj92.tistory.com/577', 'content': 'Mar 8, 2020 · 이제 추천 시스템을 위한 딥러닝 모델을 만들어줍니다. 딥러닝 모델은 저는 파이썬의 Keras(Python Keras)를 사용했습니다. 그리고 그 모델의 형태는\xa0...'}, {'url': 'https://kmhana.tistory.com/37', 'content': 'Feb 15, 2022 · 추천 시스템에서의 딥러닝 - SOTA · MLP · Autoencoder · Convolutional Neural Network · Graph CNNs · Attention 기반 추천 시스템. 특징. - Attention\xa0...'}, {'url': 'https://jalynne-kim.medium.com/%EC%B6%94%EC%B2%9C%EB%AA%A8%EB%